In [2]:
from keras.models import load_model
import tensorflow as tf
import os 
import os.path as osp
from keras import backend as K

input_path = 'model'
weight_file = '3DCNN+3LSTM_128_4.h5'
weight_file_path = osp.join(input_path,weight_file)
output_graph_name = weight_file[:-3] + '.pb'
def h5_to_pb(h5_model,output_dir,model_name,out_prefix = "output_",log_tensorboard = True):
    if osp.exists(output_dir) == False:
        os.mkdir(output_dir)
    out_nodes = []
    for i in range(len(h5_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(h5_model.output[i],out_prefix + str(i + 1))
    sess = K.get_session()
    from tensorflow.python.framework import graph_util,graph_io
    init_graph = sess.graph.as_graph_def()
    main_graph = graph_util.convert_variables_to_constants(sess,init_graph,out_nodes)
    graph_io.write_graph(main_graph,output_dir,name = model_name,as_text = False)
    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard
        import_pb_to_tensorboard.import_to_tensorboard(osp.join(output_dir,model_name),output_dir)
#输出路径
output_dir = osp.join(os.getcwd(),"trans_model")
#加载模型
h5_model = load_model(weight_file_path)
h5_to_pb(h5_model,output_dir = output_dir,model_name = output_graph_name)

print('model saved')

Using TensorFlow backend.
C:\Users\dpadmin\Anaconda3\envs\pycv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dpadmin\Anaconda3\envs\pycv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dpadmin\Anaconda3\envs\pycv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dpadmin\Anaconda3\envs\pycv\lib\sit

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 22 variables.
INFO:tensorflow:Converted 22 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.
Model Imported. Visualize by running: tensorboard --logdir=C:\GitHub\funboxing-opencv\trans_model
model saved


In [10]:
import tensorflow as tf
from tensorflow.python.platform import gfile
 
def load_pb(pb_file_path):
    sess = tf.Session()
    with gfile.FastGFile(pb_file_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')
 
    print(sess.run('b:0'))
    #输入
    input_x = sess.graph.get_tensor_by_name('x:0')
    input_y = sess.graph.get_tensor_by_name('y:0')
    #输出
    op = sess.graph.get_tensor_by_name('op_to_store:0')
    #预测结果
    ret = sess.run(op, {input_x: 3, input_y: 4})

new_path = 'trans_model'
pb_file_path = osp.join(new_path,weight_file)

load_pb(pb_file_path)

NotFoundError: NewRandomAccessFile failed to Create/Open: trans_model\3DCNN+3LSTM_128_4.h5 : The system cannot find the file specified.
; No such file or directory

In [11]:
os.getcwd()

'C:\\GitHub\\funboxing-opencv'

In [1]:
from keras.models import load_model, model_from_json

json_file = open("model/model.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
# load weights into new model
model.load_weights("model/model.h5")

print(model.outputs)
print(model.inputs)

Using TensorFlow backend.


[<tf.Tensor 'activation_4/Softmax:0' shape=(?, 3) dtype=float32>]
[<tf.Tensor 'conv3d_14_input:0' shape=(?, 1, 128, 128, 25) dtype=float32>]


In [2]:
from keras import backend as K
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session,input_graph_def,output_names,freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [1]:
# Save to ./model/tf_model.pb
# tf.train.write_graph(frozen_graph, "model", "tf_model.pb")

In [ ]:
make_cv2_compatible('model/model.pbtxt')

In [8]:
import numpy as np
import tensorflow as tf
from keras.models import load_model

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph


# X = np.array([[0,0], [0,1], [1,0], [1,1]], 'float32')
# Y = np.array([[0], [1], [1], [0]], 'float32')

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Dense(64, input_dim=2, activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])

# model.fit(X, Y, batch_size=1, nb_epoch=100, verbose=0)

# # inputs:  ['dense_input']
# print('inputs: ', [input.op.name for input in model.inputs])

# # outputs:  ['dense_4/Sigmoid']
# print('outputs: ', [output.op.name for output in model.outputs])

# model.save('./xor.h5')

model = load_model('model/model.h5')


frozen_graph = freeze_session(tf.keras.backend.get_session(), output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, './', 'tmodel.pbtxt', as_text=True)
tf.train.write_graph(frozen_graph, './', 'tmodel.pb', as_text=False)

ValueError: Unknown layer:name

In [4]:
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('./xor.h5')

# 0 ^ 0 =  [[0.01974997]]
print('0 ^ 0 = ', model.predict(np.array([[0, 0]])))

# 0 ^ 1 =  [[0.99141496]]
print('0 ^ 1 = ', model.predict(np.array([[0, 1]])))

# 1 ^ 0 =  [[0.9897714]]
print('1 ^ 0 = ', model.predict(np.array([[1, 0]])))

# 1 ^ 1 =  [[0.00406971]]
print('1 ^ 1 = ', model.predict(np.array([[1, 1]])))

0 ^ 0 =  [[0.01113283]]
0 ^ 1 =  [[0.9916477]]
1 ^ 0 =  [[0.9882267]]
1 ^ 1 =  [[0.00744604]]


In [9]:
import numpy as np
import cv2 as cv


model = cv.dnn.readNetFromTensorflow('model\\tf_model.pb')

# # 0 ^ 0 =  [[0.01974997]]
# model.setInput(np.array([[0, 0]]), name='dense_input')
# print('0 ^ 0 = ', model.forward(outputName='dense_4/Sigmoid'))

# # 0 ^ 1 =  [[0.99141496]]
# model.setInput(np.array([[0, 1]]), name='dense_input')
# print('0 ^ 1 = ', model.forward(outputName='dense_4/Sigmoid'))

# # 1 ^ 0 =  [[0.9897714]]
# model.setInput(np.array([[1, 0]]), name='dense_input')
# print('1 ^ 0 = ', model.forward(outputName='dense_4/Sigmoid'))

# # 1 ^ 1 =  [[0.00406971]]
# model.setInput(np.array([[1, 1]]), name='dense_input')
# print('1 ^ 1 = ', model.forward(outputName='dense_4/Sigmoid'))

error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\dnn\src\tensorflow\tf_importer.cpp:544: error: (-2:Unspecified error) Input layer not found: conv3d_14/bias in function 'cv::dnn::dnn4_v20190902::`anonymous-namespace'::TFImporter::connect'
